# **1. Library Import**

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda --version

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:23
🔁 Restarting kernel...
conda 22.9.0


In [1]:
!conda create --name mlops-tfx python==3.9.15

Solving environment: / - done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/mlops-tfx

  added / updated specs:
    - python==3.9.15


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    python-3.9.15              |hba424b6_0_cpython        21.0 MB  conda-forge
    setuptools-65.6.3          |     pyhd8ed1ab_0         619 KB  conda-forge
    tzdata-2022g               |       h191b570_0         106 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        21.8 MB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      con

In [2]:
!conda activate mlops-tfx


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [3]:
!pip install jupyter scikit-learn tensorflow tfx==1.11.0 flask joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.5/446.5 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [4]:
import os
import pandas as pd

import tensorflow as tf
import tensorflow_model_analysis as tfma

from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator
from tfx.components import Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

# **2. Set Pipeline Variable**

In [5]:
PIPELINE_NAME = 'disaster-tweets-pipeline'
SCHEMA_PIPELINE_NAME = 'disaster-tweets-tfdv-schema'

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# **3. Data Loading**

## 3.1 Environment and Kaggle Credential

Set up the [Colab](https://colab.research.google.com) `operating system` environment with the `KAGGLE_USERNAME` variable and the `KAGGLE_KEY` variable to connect to the [Kaggle](https://kaggle.com) platform using [Kaggle's Beta API](https://www.kaggle.com/docs/api) Token.

In [6]:
os.environ['KAGGLE_USERNAME'] = 'andrewbjamesie'
os.environ['KAGGLE_KEY']      = 'b75c236a492526b84d0dc7517c37d48b'

## 3.2 Dataset Download

Download the dataset form Kaggle with the dataset file name, `train_data_cleaning.csv`. The dataset used in this project is the [NLP with Disaster Tweets](https://www.kaggle.com/datasets/vbmokin/nlp-with-disaster-tweets-cleaning-data) dataset in the form of a `.csv` ([Comma-separated Values](https://en.wikipedia.org/wiki/Comma-separated_values)) file.

In [7]:
!kaggle datasets download -d vbmokin/nlp-with-disaster-tweets-cleaning-data -f train_data_cleaning.csv

  0% 0.00/899k [00:00<?, ?B/s]
100% 899k/899k [00:00<00:00, 18.6MB/s]


## 3.3 Dataset Preparation

In [8]:
DATA_PATH = 'data'

df = pd.read_csv('train_data_cleaning.csv')
df = df.drop(['id', 'keyword', 'location'], axis=1)
df = df.rename(columns={'target': 'label'})

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

df.to_csv(os.path.join(DATA_PATH, 'disaster_tweets.csv'), index=False)

df.head()

text  label
0  Our Deeds are the Reason of this  # earthquake...      1
1           Forest fire near La Ronge Sask .  Canada      1
2  All residents asked to  ' shelter in place '  ...      1
3  13,000 people receive  # wildfires evacuation ...      1
4  Just got sent this photo from Ruby  # Alaska a...      1

In [9]:
DATA_ROOT = 'data'

In [10]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# **4. Data Ingestion**

In [11]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=9),
        example_gen_pb2.SplitConfig.Split(name='eval',  hash_buckets=1)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [12]:
interactive_context.run(example_gen)

/usr/local/lib/python3.8/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

# **5. Data Validation**

## 5.1 Statistic Summary

In [13]:
statistics_gen = StatisticsGen(
    examples = example_gen.outputs['examples']
)

interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [14]:
interactive_context.show(statistics_gen.outputs['statistics'])

## 5.2 Data Schema

In [15]:
schema_gen = SchemaGen(
    statistics = statistics_gen.outputs['statistics']
)

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [16]:
interactive_context.show(schema_gen.outputs['schema'])

Type  Presence Valency Domain
Feature name                                
'label'         INT  required              -
'text'        BYTES  required              -

## 5.3 Data Anomaly Identification

In [17]:
example_validator = ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema     = schema_gen.outputs['schema']
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [18]:
interactive_context.show(example_validator.outputs['anomalies'])

# **6. Data Preprocessing**

In [19]:
TRANSFORM_MODULE_FILE = 'disaster_tweets_transform.py'

In [20]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf

LABEL_KEY   = "label"
FEATURE_KEY = "text"

# Renaming transformed features
def transformed_name(key):
    return key + "_xf"

# Preprocess input features into transformed features
def preprocessing_fn(inputs):
    """
    inputs:  map from feature keys to raw features
    outputs: map from feature keys to transformed features
    """

    outputs = {}
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    outputs[transformed_name(LABEL_KEY)]   = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Writing disaster_tweets_transform.py


In [21]:
transform = Transform(
    examples    = example_gen.outputs['examples'],
    schema      = schema_gen.outputs['schema'],
    module_file = os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines/disaster-tweets-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/6073844757fb4ac2b7115b69c906cf52/assets


INFO:tensorflow:Assets written to: pipelines/disaster-tweets-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/6073844757fb4ac2b7115b69c906cf52/assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

# **7. Hyperparameter Tuning**

References

- [Keras - KerasTuner](https://keras.io/keras_tuner)  
- [TensorFlow - TFX Tuner](https://www.tensorflow.org/tfx/guide/tuner)  
- [Dicoding - Pembuatan Komponen Tuner](https://www.dicoding.com/academies/443/discussions/211900)  

In [22]:
TUNER_MODULE_FILE = 'disaster_tweets_tuner.py'

In [23]:
%%writefile {TUNER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from keras_tuner.engine import base_tuner
from typing import NamedTuple, Dict, Text, Any

LABEL_KEY   = 'label'
FEATURE_KEY = 'text'

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size   = batch_size,
        features     = transform_feature_spec,
        reader       = gzip_reader_fn,
        num_epochs   = num_epochs,
        label_key    = transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE      = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize            = 'lower_and_strip_punctuation',
    max_tokens             = VOCAB_SIZE,
    output_mode            = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build keras tuner model"""
    embedding_dim = hp.Int('embedding_dim', min_value=16, max_value=128, step=16)
    lstm_units    = hp.Int('lstm_units', min_value=16, max_value=128, step=16)
    num_layers    = hp.Choice('num_layers', values=[1, 2, 3])
    dense_units   = hp.Int('dense_units', min_value=16, max_value=128, step=16)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)
    for _ in range(num_layers):
        x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer = tf.keras.optimizers.Adam(learning_rate),
        metrics   = [tf.keras.metrics.BinaryAccuracy()]
    )
    
    model.summary()
    return model

TunerFnResult = NamedTuple('TunerFnResult', [
    ('tuner', base_tuner.BaseTuner),
    ('fit_kwargs', Dict[Text, Any]),
])

early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor  = 'val_binary_accuracy',
    mode     = 'max',
    verbose  = 1,
    patience = 10
)

def tuner_fn(fn_args: FnArgs) -> None:
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, 10)
    val_set   = input_fn(fn_args.eval_files[0],  tf_transform_output, 10)

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )
    
    # Build the model tuner
    model_tuner = kt.Hyperband(
        hypermodel   = lambda hp: model_builder(hp),
        objective    = kt.Objective('val_binary_accuracy', direction='max'),
        max_epochs   = 10,
        factor       = 3,
        directory    = fn_args.working_dir,
        project_name = 'disaster_tweets_kt'
    )

    return TunerFnResult(
        tuner      = model_tuner,
        fit_kwargs = {
            'callbacks'        : [early_stop_callback],
            'x'                : train_set,
            'validation_data'  : val_set,
            'steps_per_epoch'  : fn_args.train_steps,
            'validation_steps' : fn_args.eval_steps
        }
    )

Writing disaster_tweets_tuner.py


In [24]:
tuner = Tuner(
    module_file     = os.path.abspath(TUNER_MODULE_FILE),
    examples        = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(tuner)

Trial 30 Complete [00h 03m 16s]
val_binary_accuracy: 0.7791563272476196

Best val_binary_accuracy So Far: 0.8349875807762146
Total elapsed time: 00h 41m 50s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines/disaster-tweets-pipeline/.temp/6/disaster_tweets_kt
Showing 10 best trials
Trial summary
Hyperparameters:
embedding_dim: 16
lstm_units: 112
num_layers: 3
dense_units: 64
dropout_rate: 0.5
learning_rate: 0.0001
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.8349875807762146
Trial summary
Hyperparameters:
embedding_dim: 96
lstm_units: 16
num_layers: 1
dense_units: 96
dropout_rate: 0.4
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.8300248384475708
Trial summary
Hyperparameters:
embedding_dim: 80
lstm_units: 16
num_layers: 1
dense_units: 32
dropout_rate: 0.2
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8300248384475708
Trial summary
Hyperparameters:
embedding_dim: 16
lstm_units: 112
num_layers: 3
dense_units: 64
dropout_rate: 0.5
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
t

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

# **8. Model Development**

In [25]:
TRAINER_MODULE_FILE = 'disaster_tweets_trainer.py'

In [26]:
%%writefile {TRAINER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY   = 'label'
FEATURE_KEY = 'text'

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size   = batch_size,
        features     = transform_feature_spec,
        reader       = gzip_reader_fn,
        num_epochs   = num_epochs,
        label_key    = transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence
VOCAB_SIZE      = 10000
SEQUENCE_LENGTH = 100
embedding_dim   = 16

vectorize_layer = layers.TextVectorization(
    standardize            = 'lower_and_strip_punctuation',
    max_tokens             = VOCAB_SIZE,
    output_mode            = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp['embedding_dim'], name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(hp['lstm_units']))(x)
    for _ in range(hp['num_layers']):
        x = layers.Dense(hp['dense_units'], activation='relu')(x)
    x = layers.Dropout(hp['dropout_rate'])(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer = tf.keras.optimizers.Adam(hp['learning_rate']),
        metrics   = [tf.keras.metrics.BinaryAccuracy()]
    )
    
    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        parsed_features      = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    hp      = fn_args.hyperparameters['values']
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor  = 'val_binary_accuracy',
        mode     = 'max',
        verbose  = 1,
        patience = 10
    )

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor        = 'val_binary_accuracy',
        mode           = 'max',
        verbose        = 1,
        save_best_only = True
    )

    callbacks = [
        tensorboard_callback,
        early_stop_callback,
        model_checkpoint_callback
    ]
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, hp['tuner/epochs'])
    val_set   = input_fn(fn_args.eval_files,  tf_transform_output, hp['tuner/epochs'])

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )
    
    # Build the model
    model = model_builder(hp)
    
    # Train the model
    model.fit(
        x                = train_set,
        validation_data  = val_set,
        callbacks        = callbacks,
        steps_per_epoch  = fn_args.train_steps,
        validation_steps = fn_args.eval_steps,
        epochs           = hp['tuner/epochs']
    )

    signatures = {
        'serving_default': _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(
                shape = [None],
                dtype = tf.string,
                name  = 'examples'
            )
        )
    }

    model.save(
        fn_args.serving_model_dir,
        save_format = 'tf',
        signatures  = signatures
    )

Writing disaster_tweets_trainer.py


In [27]:
trainer = Trainer(
    module_file     = os.path.abspath(TRAINER_MODULE_FILE),
    examples        = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    hyperparameters = tuner.outputs['best_hyperparameters'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape_1 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 224)              115584    
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 64)                1440

INFO:tensorflow:Assets written to: pipelines/disaster-tweets-pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/disaster-tweets-pipeline/Trainer/model/7/Format-Serving/assets


426/426 [==============================] - 33s 61ms/step - loss: 0.6497 - binary_accuracy: 0.6146 - val_loss: 0.4884 - val_binary_accuracy: 0.7978
Epoch 2/4
423/426 [============================>.] - ETA: 0s - loss: 0.4516 - binary_accuracy: 0.8020
Epoch 2: val_binary_accuracy improved from 0.79777 to 0.82878, saving model to pipelines/disaster-tweets-pipeline/Trainer/model/7/Format-Serving


INFO:tensorflow:Assets written to: pipelines/disaster-tweets-pipeline/Trainer/model/7/Format-Serving/assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines/disaster-tweets-pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/disaster-tweets-pipeline/Trainer/model/7/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

# **9. Model Analysis and Validation**

## 9.1 Resolver

In [28]:
model_resolver = Resolver(
    strategy_class = LatestBlessedModelStrategy,
    model          = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

## 9.2 Evaluator

In [29]:
eval_config = tfma.EvalConfig(
    model_specs   = [tfma.ModelSpec(label_key = 'label')],
    slicing_specs = [tfma.SlicingSpec()],
    metrics_specs = [
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name = 'ExampleCount'),
            tfma.MetricConfig(class_name = 'AUC'),
            tfma.MetricConfig(class_name = 'FalsePositives'),
            tfma.MetricConfig(class_name = 'TruePositives'),
            tfma.MetricConfig(class_name = 'FalseNegatives'),
            tfma.MetricConfig(class_name = 'TrueNegatives'),
            tfma.MetricConfig(class_name = 'BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold = tfma.GenericValueThreshold(
                        lower_bound = {'value': 0.5}
                    ),
                    change_threshold = tfma.GenericChangeThreshold(
                        direction = tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute  = {'value': 0.0001}
                    )
                )
            )
        ])
    ]
)

In [30]:
evaluator = Evaluator(
    examples       = example_gen.outputs['examples'],
    model          = trainer.outputs['model'],
    baseline_model = model_resolver.outputs['model'],
    eval_config    = eval_config
)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [31]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

## **10. Model Deployment and Export**

In [32]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model            = trainer.outputs['model'],
    model_blessing   = evaluator.outputs['blessing'],
    push_destination = pusher_pb2.PushDestination(
        filesystem = pusher_pb2.PushDestination.Filesystem(
            base_directory = 'serving_model_dir/disaster-tweets-model')
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

In [33]:
!zip -r pipelines.zip pipelines/
!zip -r serving_model_dir.zip serving_model_dir/
!pip freeze > requirements.txt

  adding: pipelines/ (stored 0%)
  adding: pipelines/disaster-tweets-pipeline/ (stored 0%)
  adding: pipelines/disaster-tweets-pipeline/Transform/ (stored 0%)
  adding: pipelines/disaster-tweets-pipeline/Transform/pre_transform_stats/ (stored 0%)
  adding: pipelines/disaster-tweets-pipeline/Transform/pre_transform_stats/5/ (stored 0%)
  adding: pipelines/disaster-tweets-pipeline/Transform/pre_transform_stats/5/FeatureStats.pb (deflated 54%)
  adding: pipelines/disaster-tweets-pipeline/Transform/transform_graph/ (stored 0%)
  adding: pipelines/disaster-tweets-pipeline/Transform/transform_graph/5/ (stored 0%)
  adding: pipelines/disaster-tweets-pipeline/Transform/transform_graph/5/transformed_metadata/ (stored 0%)
  adding: pipelines/disaster-tweets-pipeline/Transform/transform_graph/5/transformed_metadata/schema.pbtxt (deflated 55%)
  adding: pipelines/disaster-tweets-pipeline/Transform/transform_graph/5/transform_fn/ (stored 0%)
  adding: pipelines/disaster-tweets-pipeline/Transform/tr